# Attention Toxicity Model

This notebook trains a version of the toxicity model that uses attention. It uses a CNN architecture for text classification trained on the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973) and pre-trained GloVe embeddings which can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/).

This model is a modification of [example code](https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py) found in the [Keras Github repository](https://github.com/fchollet/keras) and released under an [MIT license](https://github.com/fchollet/keras/blob/master/LICENSE). For further details of this license, find it [online](https://github.com/fchollet/keras/blob/master/LICENSE) or in this repository in the file KERAS_LICENSE. 

## Usage Instructions

Prior to running the notebook, you must:

* Download the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973)
* Download pre-trained [GloVe embeddings](http://nlp.stanford.edu/data/glove.6B.zip)
* (optional) To skip the training step, you will need to download a model and tokenizer file. We are looking into the appropriate means for distributing these (sometimes large) files.

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd

from model_tool import *

/home/jb/repos/unintended-ml-bias-analysis/venv/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


HELLO from model_tool


In [3]:
from model_tool import (DEFAULT_EMBEDDINGS_PATH,
DEFAULT_MODEL_DIR, DEFAULT_HPARAMS)
from keras.utils import to_categorical
from keras.layers import Multiply
from keras.callbacks import TensorBoard
from time import time


class AttentionToxModel(ToxModel):
    def __init__(self,
                 model_name=None,
                 model_dir=DEFAULT_MODEL_DIR,
                 embeddings_path=DEFAULT_EMBEDDINGS_PATH,
                 hparams=None):
        self.model_dir = model_dir
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.hparams = DEFAULT_HPARAMS.copy()
        self.embeddings_path = embeddings_path
        if hparams:
            self.update_hparams(hparams)
        if model_name:
            self.load_model_from_name(model_name)
            self.load_probs_model(model_name)
        self.print_hparams()

    def load_probs_model(self, model_name):
        probs_model_name = model_name + "_probs"
        self.probs_model = load_model(
                os.path.join(
                    self.model_dir, '%s_model.h5' % probs_model_name))

    def save_prob_model(self):
        self.probs_model_name = self.model_name + "probs"

    def build_dense_attention_layer(self, input_tensor):
        # softmax
        attention_probs = Dense(self.hparams['max_sequence_length'],
                                activation='softmax',
                                name='attention_vec')(input_tensor)
        # context vector
        attention_mul = Multiply()([input_tensor, attention_probs])
        return {'attention_probs': attention_probs,
                'attention_preds': attention_mul}

    def train(
                self,
                training_data_path,
                validation_data_path, text_column,
                label_column, model_name):
        self.model_name = model_name
        self.save_hparams(model_name)

        train_data = pd.read_csv(training_data_path)
        valid_data = pd.read_csv(validation_data_path)

        print('Fitting tokenizer...')
        self.fit_and_save_tokenizer(train_data[text_column])
        print('Tokenizer fitted!')

        print('Preparing data...')
        train_text, train_labels = (self.prep_text(train_data[text_column]),
                                    to_categorical(train_data[label_column]))
        valid_text, valid_labels = (self.prep_text(valid_data[text_column]),
                                    to_categorical(valid_data[label_column]))
        print('Data prepared!')

        print('Loading embeddings...')
        self.load_embeddings()
        print('Embeddings loaded!')

        print('Building model graph...')
        self.build_model()
        print('Training model...')
        tensorboard = TensorBoard(log_dir="logs/{}".format(time()), write_graph=True)

        preds_save_path = os.path.join(
                            self.model_dir, '%s_model.h5' % self.model_name)
        probs_save_path = os.path.join(
                            self.model_dir, '%s_probs_model.h5'
                            % self.model_name)
        preds_callbacks = [ModelCheckpoint(
                            preds_save_path,
                            save_best_only=True,
                            verbose=self.hparams['verbose']), tensorboard]
        probs_callbacks = [ModelCheckpoint(
                            probs_save_path,
                            save_best_only=True,
                            verbose=self.hparams['verbose'])]

        if self.hparams['stop_early']:
            early_stop = EarlyStopping(
                            min_delta=self.hparams['es_min_delta'],
                            monitor='val_loss',
                            patience=self.hparams['es_patience'],
                            verbose=self.hparams['verbose'], mode='auto')
            probs_callbacks.append(early_stop)
            preds_callbacks.append(early_stop)

        self.model.fit(train_text,
                       train_labels,
                       batch_size=self.hparams['batch_size'],
                       epochs=self.hparams['epochs'],
                       validation_data=(valid_text, valid_labels),
                       callbacks=preds_callbacks,
                       verbose=2)

        print('Model trained!')
        print('Best model saved to {}'.format(preds_save_path))
        print('Fitting probs model')

        self.probs_model.fit(
                    train_text,
                    train_labels,
                    batch_size=self.hparams['batch_size'],
                    epochs=self.hparams['epochs'],
                    validation_data=(valid_text, valid_labels),
                    callbacks=probs_callbacks,
                    verbose=2)
        self.probs_model = load_model(probs_save_path)
        print('Loading best model from checkpoint...')
        self.model = load_model(preds_save_path)
        print('Model loaded!')

    def build_model(self):
        print('print inside build model')
        sequence_input = Input(
                            shape=(self.hparams['max_sequence_length'],),
                            dtype='int32')
        embedding_layer = Embedding(
                            len(self.tokenizer.word_index) + 1,
                            self.hparams['embedding_dim'],
                            weights=[self.embedding_matrix],
                            input_length=self.hparams['max_sequence_length'],
                            trainable=self.hparams['embedding_trainable'])

        embedded_sequences = embedding_layer(sequence_input)
        x = embedded_sequences
        for filter_size, kernel_size, pool_size in zip(
                self.hparams['cnn_filter_sizes'],
                self.hparams['cnn_kernel_sizes'],
                self.hparams['cnn_pooling_sizes']):
            x = self.build_conv_layer(x, filter_size, kernel_size, pool_size)

        x = Flatten()(x)
        x = Dropout(self.hparams['dropout_rate'], name="Dropout")(x)
        x = Dense(self.hparams['max_sequence_length'], activation='relu', name="Dense_RELU")(x)

        # build prediction model
        attention_dict = self.build_dense_attention_layer(x)
        preds = attention_dict['attention_preds']
        preds = Dense(2, name="preds_dense", activation='softmax')(preds)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.model = Model(sequence_input, preds)
        self.model.compile(
                loss='categorical_crossentropy',
                optimizer=rmsprop,
                metrics=['acc'])

        # now make probs model
        probs = attention_dict['attention_probs']
        probs = Dense(2, name='probs_dense')(probs)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.probs_model = Model(sequence_input, preds)
        self.probs_model.compile(
                loss='mse', optimizer=rmsprop, metrics=['acc'])
        # build probabilities model
        self.save_prob_model()


## Load Data

In [4]:
SPLITS = ['train', 'dev', 'test']

wiki = {}
debias = {}
random = {}
for split in SPLITS:
    wiki[split] = '../data/wiki_%s.csv' % split
    debias[split] = '../data/wiki_debias_%s.csv' % split
    random[split] = '../data/wiki_debias_random_%s.csv' % split

## Train Models

In [5]:
hparams = {'epochs': 20}

### Random attention model

In [6]:
model_names = ['test_atn_cnn_random_tox_v4_{}'.format(i) for i in xrange(100, 110)]
for model_name in model_names:
    MODEL_NAME = model_name
    random_model = AttentionToxModel(hparams=hparams)
    random_model.train(random['train'], random['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.22855, saving model to ../models/test_atn_cnn_random_tox_v4_100_model.h5
11s - loss: 0.2938 - acc: 0.9055 - val_loss: 0.2286 - val_acc: 0.9078
Epoch 2/20
Epoch 00001: val_loss improved from 0.22855 to 0.18390, saving model to ../models/test_atn_cnn_random_tox_v4_100_model.h5
10s - loss: 0.2086 - acc: 0.9202 - val_loss: 0.1839 - val_acc: 0.9347
Epoch 3/20


KeyboardInterrupt: 

In [6]:
random_test = pd.read_csv(random['test'])
random_model.score_auc(random_test['comment'], random_test['is_toxic'])

0.963784838550671

### Plain attention wikipedia model

In [9]:
model_names = ['atn_cnn_wiki_tox_v4_{}'.format(i) for i in xrange(100, 110)]
for model_name in model_names:
    MODEL_NAME = model_name
    wiki_model = AttentionToxModel(hparams=hparams)
    wiki_model.train(wiki['train'], wiki['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 95692 samples, validate on 32128 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.24035, saving model to ../models/atn_cnn_wiki_tox_v4_100_model.h5
12s - loss: 0.3070 - acc: 0.9032 - val_loss: 0.2403 - val_acc: 0.9045
Epoch 2/20
Epoch 00001: val_loss improved from 0.24035 to 0.19245, saving model to ../models/atn_cnn_wiki_tox_v4_100_model.h5
11s - loss: 0.2164 - acc: 0.9149 - val_loss: 0.1925 - val_acc: 0.9316
Epoch 3/20
Epoch 00002: val_loss impr

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 95692 samples, validate on 32128 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.23601, saving model to ../models/atn_cnn_wiki_tox_v4_102_model.h5
13s - loss: 0.3040 - acc: 0.9019 - val_loss: 0.2360 - val_acc: 0.9045
Epoch 2/20
Epoch 00001: val_loss improved from 0.23601 to 0.18950, saving model to ../models/atn_cnn_wiki_tox_v4_102_model.h5
11s - loss: 0.2094 - acc: 0.9168 - val_loss: 0.1895 - val_acc: 0.9335
Epoch 3/20
Epoch 00002: val_loss improved from 0.18950 to 0.16309, saving model to ../models/atn_cnn_wiki_tox_v4_102_model.h5
11s - loss: 0.1752 - acc: 0.9362 - val_loss: 0.1631 - val_acc: 0.9410
Epoch 4/20
Epoch 00003: val_loss improved from 0.16309 to 0.15577, saving model to ../models/atn_cnn_wiki_tox_v4_102_model.h5
11s - loss: 0.1599 - acc: 0.9411 - val_loss: 0.1558 - val_ac

Epoch 00002: val_loss improved from 0.18524 to 0.16163, saving model to ../models/atn_cnn_wiki_tox_v4_104_model.h5
11s - loss: 0.1755 - acc: 0.9361 - val_loss: 0.1616 - val_acc: 0.9409
Epoch 4/20
Epoch 00003: val_loss improved from 0.16163 to 0.15645, saving model to ../models/atn_cnn_wiki_tox_v4_104_model.h5
11s - loss: 0.1583 - acc: 0.9416 - val_loss: 0.1564 - val_acc: 0.9429
Epoch 5/20
Epoch 00004: val_loss improved from 0.15645 to 0.14141, saving model to ../models/atn_cnn_wiki_tox_v4_104_model.h5
11s - loss: 0.1464 - acc: 0.9458 - val_loss: 0.1414 - val_acc: 0.9466
Epoch 6/20
Epoch 00005: val_loss improved from 0.14141 to 0.14015, saving model to ../models/atn_cnn_wiki_tox_v4_104_model.h5
11s - loss: 0.1378 - acc: 0.9492 - val_loss: 0.1401 - val_acc: 0.9489
Epoch 7/20
Epoch 00006: val_loss improved from 0.14015 to 0.13171, saving model to ../models/atn_cnn_wiki_tox_v4_104_model.h5
11s - loss: 0.1315 - acc: 0.9516 - val_loss: 0.1317 - val_acc: 0.9494
Epoch 8/20
Epoch 00007: val_los

Epoch 00009: val_loss improved from 0.14354 to 0.13993, saving model to ../models/atn_cnn_wiki_tox_v4_106_model.h5
11s - loss: 0.1304 - acc: 0.9517 - val_loss: 0.1399 - val_acc: 0.9498
Epoch 11/20
Epoch 00010: val_loss did not improve
11s - loss: 0.1252 - acc: 0.9537 - val_loss: 0.1552 - val_acc: 0.9476
Epoch 12/20
Epoch 00011: val_loss improved from 0.13993 to 0.13845, saving model to ../models/atn_cnn_wiki_tox_v4_106_model.h5
11s - loss: 0.1205 - acc: 0.9554 - val_loss: 0.1384 - val_acc: 0.9517
Epoch 13/20
Epoch 00012: val_loss improved from 0.13845 to 0.13558, saving model to ../models/atn_cnn_wiki_tox_v4_106_model.h5
11s - loss: 0.1164 - acc: 0.9574 - val_loss: 0.1356 - val_acc: 0.9487
Epoch 14/20
Epoch 00013: val_loss did not improve
11s - loss: 0.1127 - acc: 0.9587 - val_loss: 0.1384 - val_acc: 0.9500
Epoch 15/20
Epoch 00014: val_loss did not improve
11s - loss: 0.1087 - acc: 0.9598 - val_loss: 0.1631 - val_acc: 0.9384
Epoch 00014: early stopping
Model trained!
Best model saved t

Train on 95692 samples, validate on 32128 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.03462, saving model to ../models/atn_cnn_wiki_tox_v4_108_probs_model.h5
12s - loss: 0.0329 - acc: 0.9567 - val_loss: 0.0346 - val_acc: 0.9546
Epoch 2/20
Epoch 00001: val_loss did not improve
11s - loss: 0.0314 - acc: 0.9595 - val_loss: 0.0389 - val_acc: 0.9468
Epoch 3/20
Epoch 00002: val_loss improved from 0.03462 to 0.03425, saving model to ../models/atn_cnn_wiki_tox_v4_108_probs_model.h5
11s - loss: 0.0305 - acc: 0.9611 - val_loss: 0.0342 - val_acc: 0.9549
Epoch 4/20
Epoch 00003: val_loss did not improve
11s - loss: 0.0292 - acc: 0.9632 - val_loss: 0.0385 - val_acc: 0.9471
Epoch 5/20
Epoch 00004: val_loss did not improve
11s - loss: 0.0279 - acc: 0.9649 - val_loss: 0.0396 - val_acc: 0.9472
Epoch 00004: early stopping
Loading best model from checkpoint...
Model loaded!
Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40

In [12]:
wiki_test = pd.read_csv(wiki['test'])
wiki_model.score_auc(wiki_test['comment'], wiki_test['is_toxic'])

0.9599183439957785

### Debiased attention model

In [14]:
model_names = ['atn_cnn_debias_tox_v4_{}'.format(i) for i in xrange(100, 110)]
for model_name in model_names:
    MODEL_NAME = model_name
    debias_model = AttentionToxModel(hparams=hparams)
    debias_model.train(debias['train'], debias['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.23235, saving model to ../models/atn_cnn_debias_tox_v4_100_model.h5
15s - loss: 0.2965 - acc: 0.9065 - val_loss: 0.2324 - val_acc: 0.9078
Epoch 2/20
Epoch 00001: val_loss improved from 0.23235 to 0.19087, saving model to ../models/atn_cnn_debias_tox_v4_100_model.h5
12s - loss: 0.2108 - acc: 0.9180 - val_loss: 0.1909 - val_acc: 0.9336
Epoch 3/20
Epoch 00002: val_loss 

Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.23285, saving model to ../models/atn_cnn_debias_tox_v4_102_model.h5
15s - loss: 0.2949 - acc: 0.9053 - val_loss: 0.2329 - val_acc: 0.9078
Epoch 2/20
Epoch 00001: val_loss improved from 0.23285 to 0.18998, saving model to ../models/atn_cnn_debias_tox_v4_102_model.h5
12s - loss: 0.2135 - acc: 0.9183 - val_loss: 0.1900 - val_acc: 0.9332
Epoch 3/20
Epoch 00002: val_loss improved from 0.18998 to 0.16787, saving model to ../models/atn_cnn_debias_tox_v4_102_model.h5
12s - loss: 0.1821 - acc: 0.9361 - val_loss: 0.1679 - val_acc: 0.9403
Epoch 4/20
Epoch 00003: val_loss improved from 0.16787 to 0.15722, saving model to ../models/atn_cnn_debias_tox_v4_102_model.h5
12s - loss: 0.1631 - acc: 0.9416 - val_loss: 0.1572 - val_acc: 0.9424
Epoch 5/20
Epoch 00004: val_loss improved from 0.15722 to 0.14233, saving model to ../models/atn_cnn_debias_tox_v4_102_model.h5
12s - loss: 0.15

Epoch 00006: val_loss improved from 0.13991 to 0.13271, saving model to ../models/atn_cnn_debias_tox_v4_104_model.h5
12s - loss: 0.1331 - acc: 0.9518 - val_loss: 0.1327 - val_acc: 0.9510
Epoch 8/20
Epoch 00007: val_loss improved from 0.13271 to 0.12807, saving model to ../models/atn_cnn_debias_tox_v4_104_model.h5
12s - loss: 0.1265 - acc: 0.9542 - val_loss: 0.1281 - val_acc: 0.9535
Epoch 9/20
Epoch 00008: val_loss did not improve
12s - loss: 0.1216 - acc: 0.9558 - val_loss: 0.1290 - val_acc: 0.9529
Epoch 10/20
Epoch 00009: val_loss did not improve
12s - loss: 0.1171 - acc: 0.9578 - val_loss: 0.1365 - val_acc: 0.9477
Epoch 00009: early stopping
Model trained!
Best model saved to ../models/atn_cnn_debias_tox_v4_104_model.h5
Fitting probs model
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.03399, saving model to ../models/atn_cnn_debias_tox_v4_104_probs_model.h5
15s - loss: 0.0313 - acc: 0.9594 - val_loss: 0.0340 - val_acc: 0.956

Epoch 00012: val_loss improved from 0.12207 to 0.12143, saving model to ../models/atn_cnn_debias_tox_v4_106_model.h5
12s - loss: 0.1096 - acc: 0.9595 - val_loss: 0.1214 - val_acc: 0.9549
Epoch 14/20
Epoch 00013: val_loss improved from 0.12143 to 0.12067, saving model to ../models/atn_cnn_debias_tox_v4_106_model.h5
12s - loss: 0.1059 - acc: 0.9609 - val_loss: 0.1207 - val_acc: 0.9552
Epoch 15/20
Epoch 00014: val_loss did not improve
12s - loss: 0.1026 - acc: 0.9621 - val_loss: 0.1353 - val_acc: 0.9457
Epoch 16/20
Epoch 00015: val_loss did not improve
12s - loss: 0.0993 - acc: 0.9638 - val_loss: 0.1238 - val_acc: 0.9559
Epoch 00015: early stopping
Model trained!
Best model saved to ../models/atn_cnn_debias_tox_v4_106_model.h5
Fitting probs model
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.03754, saving model to ../models/atn_cnn_debias_tox_v4_106_probs_model.h5
15s - loss: 0.0265 - acc: 0.9651 - val_loss: 0.0375 - val_acc: 0.9

Epoch 00005: val_loss did not improve
12s - loss: 0.0268 - acc: 0.9667 - val_loss: 0.0341 - val_acc: 0.9563
Epoch 00005: early stopping
Loading best model from checkpoint...
Model loaded!
Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.23670, saving model to ../models/atn_cnn_debias_tox_v4_109_model.h5
16s - loss: 0.3046 - acc: 0.9066 - val_loss: 0.2367 - val_acc: 0.9078
Epoch 2/20
Epoch 00001: val_loss improved f

In [15]:
debias_test = pd.read_csv(debias['test'])
debias_model.score_auc(debias_test['comment'], debias_test['is_toxic'])

0.9476482738664272